# SatAgro Klient API Metos
Marcin Gałązka, Marcin Borowiec


## Fieldclimate
- fieldclimate jest platforma internetowa stworzona do obsługi systemów monitorujących marki METOS(tzw. stacje meteorologiczne)
- systemy monitorujące są urządzeniami wyposażonymi w różnego rodzaju wyspecjalizowane sensory dokonujące pomiarów określonej wartości fizycznej środowiska(np. model iMETOS IMT80 posiada wbudowany czujnik temperatury powietrza i wskaźnik opadów )
- systemy monitorujace nawiazuja polaczenie z serwerem meteos(wykorzystujac np. sieć GSM) pobierając z niego konfiguracje oraz przesyłając zebrane dane w zaplanowanych, skonfigurowanych w fieldclimate terminach(godziny)


## Fieldclimate
imeteos imt200 | imeteos imt300
- | - 
![alt](presentation/imetos_imt200.jpg) | ![alt](presentation/imetos_imt300.jpg)

## Najważniejsze technologie i narzędzia
- asyncio(asynchronous input/output)
- oauth2, hmac(metody autoryzacja użytkowników w serwisie)
- marshmallow(narzędzie do budowania schematów modeli)



## Asyncio
- Asynchronous input/output
- Umożliwia tworzenie konkurencyjnych aplikacji przy użyciu korutyn(coroutines)
- Stosuje jednowątkowe, jednoprocesorowe podejście w którym czesci aplikacji wspolpracuja ze soba tak aby sterowanie programu przechodzi między zadaniami w odpowiednim czasie(np. blokujące operacje wejscia/wyjscia, planowanie zadań na odpowiedni czas, obsługa sygnałów itp).


## Asyncio
##### Eventloops
- główny element asyncio
- odpowiada za zmianę kontekstu między zadaniami, obsługę sygnałów i blokujących operacji
- aplikacja rejestruje zadania w eventloop przekazując odpowiedzialność za wywołanie kodu ww. zadań gdy pewne zasoby systemowe będą dostępne



## Asyncio
##### Eventloops - przykład
Serwer sieciowy otwiera sockety, rejestruje kod który powinien być wykonany gdy operacja wejścia będzie dostępna w eventloop. Eventloop zarządza zmiana kontekstu wykonywania aplikacji. W momencie, w którym pojawia się dane, które można przeczytać eventloop przenosi sterowanie do wyżej wspomnianego kodu. Po przeczytaniu danych i wykonaniu na nich potrzebnych operacji sterowanie powraca do eventloop(np. gdy kod czeka na kolejna porcje danych).



## Asyncio
##### Coroutines
- Eventloop swoje działanie opiera na korutynach
- korutyny to specjalne funkcje, które mogą przekazać kontrolę do wywołującego nie tracąc przy tym swojego stanu
- podobne do generatorów




## Asyncio
##### Coroutines - cd
- Tworzymy je przy użyciu słowa kluczowego async
- Innym ważnym słowem kluczowym jest await(odpowiednik yeld from dla generatorów). Await sygnalizuje, ze dana korutyna czeka na wynik wykonania pewnego zadania(np. innej korutyny, tasku, feature; wbudowane obiekty na ktore mozna dokonac await np. obiekty reprezentujące operacja wejscia/wyjscia itp.) 




In [13]:
import asyncio

async def sub0():
    print('0 in')
    await asyncio.sleep(10)
    print('0 out')


async def sub1():
    print('1 in')
    print('1 out')
    
async def concurrency():
    task0 = asyncio.get_event_loop().create_task(sub0())
    task1 = asyncio.get_event_loop().create_task(sub1())
    await asyncio.sleep(20)
    print('wait')
    await task0
    await task1
    print('done')
    
loop = asyncio.new_event_loop()
loop.run_until_complete(concurrency())
loop.close()

0 in
1 in
1 out
0 out
wait
done


In [6]:
import asyncio

async def sub0():
    print('0 in')
    await asyncio.sleep(10)
    print('0 out')


async def sub1():
    print('1 in')
    print('1 out')
    
async def noconcurrency():
    print('wait')
    await sub0()
    await sub1()
    print('done')
    
loop = asyncio.new_event_loop()
loop.run_until_complete(noconcurrency())
loop.close()

wait
0 in
0 out
1 in
1 out
done


In [7]:
import asyncio
from fieldclimate.connection.hmac import HMAC
from fieldclimate.connection.oauth2 import OAuth2, WebBasedProvider
import os

public_key = os.environ['FIELDCLIMATE_HAMAC_PUBLIC']
private_key = os.environ['FIELDCLIMATE_HAMAC_PRIVATE']

async def hmac():
    async with HMAC(public_key, private_key) as client:
        stations = await client.user.list_of_user_devices()
        station_id = stations.response[0]['name']['original']
        station_data = await client.data.get_last_data(station_id, 'daily', '1w', 'optimized')
        for (sensor_tag, sensor) in station_data.response['data'].items():
            print('Sensor {} has tag {} and supports the following aggregations: {}'.format(
                sensor['name'], sensor_tag, list(sensor['aggr'].keys())
            ))
    
loop = asyncio.new_event_loop()
loop.run_until_complete(hmac())
loop.close()

Sensor Wind direction has tag 3_X_X_143 and supports the following aggregations: ['avg', 'last']
Sensor Solar Panel has tag 4_X_X_30 and supports the following aggregations: ['last']
Sensor Wind speed has tag 6_X_X_5 and supports the following aggregations: ['avg', 'max']
Sensor Battery has tag 7_X_X_7 and supports the following aggregations: ['last']
Sensor Leaf Wetness has tag 8_X_X_4 and supports the following aggregations: ['time']
Sensor HC Serial Number has tag 13_X_X_508 and supports the following aggregations: ['last']
Sensor HC Air temperature has tag 14_X_X_506 and supports the following aggregations: ['avg', 'max', 'min']
Sensor HC Relative humidity has tag 15_X_X_507 and supports the following aggregations: ['avg', 'max', 'min']
Sensor Dew Point has tag 16_X_X_21 and supports the following aggregations: ['avg', 'min']
Sensor HC Serial Number has tag 17_X_X_508 and supports the following aggregations: ['last']
Sensor HC Air temperature has tag 18_X_X_506 and supports the fol

## Uwierzytelnianie hmac/oauth2

##### Hmac(hash-based message authentication code)
- specjalny typ MAC(message authentication code)
- pozwala na zagwarantowanie integralności danych jak i uwierzytelnia nadawcę
- moze wykorzystywac dowolna haszujaca funkcje kryptograficzna(np. SHA-256) 




## Uwierzytelnianie hmac/oauth2

##### Hmac - definicja(RFC 2104)
![](presentation/hmac.png)

  K - sekret  
  H - funkcja haszujaca  
  opad/ipad - pewny pad  
  m - wiadomosc




In [ ]:
from datetime import datetime

from Crypto.Hash import SHA256, HMAC as HASH_HMAC

from fieldclimate.connection.base import ConnectionBase


class HMAC(ConnectionBase):

    def __init__(self, public_key, private_key):
        self._publicKey = public_key
        self._privateKey = private_key

    def _modify_request(self, request):
        date_stamp = datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')
        request.headers['Date'] = date_stamp
        msg = '{}/{}{}{}'.format(request.method, request.route, date_stamp, self._publicKey).encode(encoding='utf-8')
        h = HASH_HMAC.new(self._privateKey.encode(encoding='utf-8'), msg, SHA256)
        signature = h.hexdigest()
        request.headers['Authorization'] = 'hmac {}:{}'.format(self._publicKey, signature)



## Uwierzytelnianie hmac/oauth2

##### Oauth2
- wiadomosci sa uwierzytelnianie za pomocą tokena dostępu(access_token)
- access_token jest dostarczany do serwera wraz z wiadomością(np. w nagłówku http)
- access_token można pozyskać na kilka sposobów(grant type) w zależności od tego co obsługuje serwis, moze sie zdarzyc, ze serwer wydający tokeny(token server) i serwis przyjmujący zapytania http to różne instancje





## Uwierzytelnianie hmac/oauth2

##### Oauth2 - przykładowy workflow
- klient aplikacji probuje uzyskac access_token od token serwer przestrzegając obsługiwanych protokołów(np. przesyła nazwę uzytkownika i haslo, inna metoda to przesyłanie pewnego pośredniego tokena - access_code)
- jeżeli wszystko jest ok token serwer zwraca petentowi access_token oraz refresh_token. Access_token ma pewien ograniczony czas życia, gdy się wyczerpie petent może uzyskać nowy access_token przy użyciu refresh_token, z pominięciem bardziej czasochłonnych ww. natywnych metod.
- klient wykonuje zapytania http do serwera wykonującego właściwe usługi(np. o podanie danych stacji) identyfikując się access_token.






In [ ]:
import asyncio
import webbrowser
from abc import ABC, abstractmethod

from aiohttp import web

from fieldclimate.connection.base import ConnectionBase
from fieldclimate.reqresp import ResponseException
from fieldclimate.tools import get_credentials

credentials = get_credentials()
client_id = credentials['client_id']
client_secret = credentials['client_secret']
auth_url = 'https://oauth.fieldclimate.com/authorize?response_type=code&client_id={}&state=xyz'.format(client_id)
ww

class AuthCodeProvider(ABC):

    @abstractmethod
    async def get_auth_code(self):
        pass


class SimpleProvider(AuthCodeProvider):
    def __init__(self, auth_code):
        self._auth_code = auth_code

    async def get_auth_code(self):
        return self._auth_code


class WebBasedProvider(AuthCodeProvider):
    default_port = 5555

    def __init__(self):
        self._port = self.default_port
        self._app = self._make_app()
        self._event = None
        self._auth_code = None

    async def _handle_get(self, request):
        self._auth_code = request.query.get('code', None)
        if self._auth_code is not None:
            self._event.set()
        return web.Response(text='Received code {}'.format(self._auth_code))

    def _make_app(self):
        app = web.Application()
        app.add_routes([web.get('/', self._handle_get)])
        app.add_routes([web.get('/oauth2/callback', self._handle_get)])
        return app

    async def get_auth_code(self):
        self._event = asyncio.Event()
        loop = asyncio.get_event_loop()
        server = await loop.create_server(self._app.make_handler(), None, self._port)
        webbrowser.open(auth_url)
        await self._event.wait()
        server.close()
        return self._auth_code


class OAuth2(ConnectionBase):
    def __init__(self, auth_code_provider):
        self._auth_code_provider = auth_code_provider
        self._access_token = None
        self._refresh_token = None

    async def _get_token(self):
        if self._refresh_token is not None:
            params = {
                'client_id': client_id,
                'client_secret': client_secret,
                'grant_type': 'refresh_token',
                'refresh_token': self._refresh_token
            }
        else:
            params = {
                'client_id': client_id,
                'client_secret': client_secret,
                'grant_type': 'authorization_code',
                'code': await self._auth_code_provider.get_auth_code()
            }
        result = await self._session.request('POST', 'https://oauth.fieldclimate.com/token', data=params)
        response = await result.json(
            content_type=None)
        if result.status >= 300:
            raise ResponseException(result.status, response)
        self._access_token = response['access_token']
        self._refresh_token = response['refresh_token']

    def _modify_request(self, request):
        request.headers['Authorization'] = 'Authorization: Bearer {}'.format(self._access_token)

    async def _make_request(self, method, route, data=None):
        if self._access_token is None:
            await self._get_token()
        try:
            response = await super()._make_request(method, route, data)
        except ResponseException as e:
            if e.code == 401:
                await self._get_token()
                response = await super()._make_request(method, route, data)
            else:
                raise
        return response


In [14]:
import asyncio
from fieldclimate.connection.hmac import HMAC
from fieldclimate.connection.oauth2 import OAuth2, WebBasedProvider

async def oauth2():
    async with OAuth2(WebBasedProvider()) as client:
        stations = await client.user.list_of_user_devices()
        station_id = stations.response[1]['name']['original']
        station_data = await client.data.get_last_data(station_id, 'monthly', '12m', 'optimized')
        dates = station_data.response['dates']
        sensor_tag = '5_X_X_6'
        unit = station_data.response['data'][sensor_tag]['unit']
        precipitation_sums = station_data.response['data'][sensor_tag]['aggr']['sum']
        for (date, precipitation_sum) in zip(dates, precipitation_sums):
            print('On month {} the precipitation sum was {} {}.'.format(
                date, precipitation_sum, unit
            ))
    
loop = asyncio.new_event_loop()
loop.run_until_complete(oauth2())
loop.close()

On month 2018-03-01 00:00:00 the precipitation sum was 205.4 mm.
On month 2018-04-01 00:00:00 the precipitation sum was 80 mm.
On month 2018-05-01 00:00:00 the precipitation sum was 77 mm.
On month 2018-06-01 00:00:00 the precipitation sum was 73.4 mm.
On month 2018-07-01 00:00:00 the precipitation sum was 27.2 mm.
On month 2018-08-01 00:00:00 the precipitation sum was 53.8 mm.
On month 2018-09-01 00:00:00 the precipitation sum was 69.6 mm.
On month 2018-10-01 00:00:00 the precipitation sum was 214.4 mm.
On month 2018-11-01 00:00:00 the precipitation sum was 239.2 mm.
On month 2018-12-01 00:00:00 the precipitation sum was 37 mm.
On month 2019-01-01 00:00:00 the precipitation sum was 47.8 mm.
On month 2019-02-01 00:00:00 the precipitation sum was 132.8 mm.


## Marshmallow

- biblioteka do serializacji/ deserializacji zlozonych obiektow z/do wbudowanych obiektow pythonowych(co jest przydatne podczas parsowania danych zapisanych w jsonie)
- podobna w dzialaniu do ORM(object relation mapping/maper)
- wykorzystuje schematy danych, ktore sa odpowiednikiem jsonschema






In [ ]:
from marshmallow import Schema, fields, post_load

from fieldclimate.models.user import UserInfo, UserSettings, User, UserCompany, UserAddress


class UserInfoSchema(Schema):
    name = fields.String(required=True)
    lastname = fields.String(required=True)
    email = fields.String(required=True)
    title = fields.String(allow_none=True)
    phone = fields.String(allow_none=True)
    cellphone = fields.String(allow_none=True)
    fax = fields.String(allow_none=True)

    @post_load
    def make_user(self, data):
        return UserInfo(**data)


class UserCompanySchema(Schema):
    name = fields.String(required=True, allow_none=True)
    profession = fields.String(allow_none=True)
    department = fields.String(allow_none=True)

    @post_load
    def make_user(self, data):
        return UserCompany(**data)


class UserAddressSchema(Schema):
    country = fields.String(required=True)
    street = fields.String(allow_none=True)
    city = fields.String(allow_none=True)
    district = fields.String(allow_none=True)
    zip = fields.String(allow_none=True)

    @post_load
    def make_user(self, data):
        return UserAddress(**data)


class UserSettingsSchema(Schema):
    language = fields.String(required=True)
    newsletter = fields.Boolean()
    # Is this required or optional?
    unit_system = fields.String(required=True)

    @post_load
    def make_user(self, data):
        return UserSettings(**data)


class UserSchema(Schema):
    username = fields.String(required=True)
    created_at = fields.String()
    created_by = fields.String()
    create_time = fields.String()
    last_access = fields.String()
    info = fields.Nested(UserInfoSchema, required=True)
    company = fields.Nested(UserCompanySchema, required=True)
    address = fields.Nested(UserAddressSchema, required=True)
    settings = fields.Nested(UserSettingsSchema, required=True)

    @post_load
    def make_user(self, data):
        return User(**data)


In [12]:
import asyncio
from fieldclimate.connection.hmac import HMAC
from fieldclimate.connection.oauth2 import OAuth2, WebBasedProvider
from fieldclimate.schemas.user import UserSchema
import os

public_key = os.environ['FIELDCLIMATE_HAMAC_PUBLIC']
private_key = os.environ['FIELDCLIMATE_HAMAC_PRIVATE']

async def schema():
    async with HMAC(public_key, private_key) as client:
        user_response = await client.user.user_information()
        response = user_response.response
        schema = UserSchema()
        result = schema.load(response)
        user = result.data
        print(type(user))
        print(user.info.name)
    
loop = asyncio.new_event_loop()
loop.run_until_complete(schema())
loop.close()

<class 'fieldclimate.models.user.User'>
Demo
